<a href="https://colab.research.google.com/github/y-arjun-y/cloudclassifer/blob/main/cloudclassifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install fastbook==0.0.17
# !pip uninstall tornado -y
# !yes | pip install tornado==5.1.0

import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 4.3 MB/s 
  Attempting uninstall: fastbook
    Found existing installation: fastbook 0.0.18
    Uninstalling fastbook-0.0.18:
      Successfully uninstalled fastbook-0.0.18
Found existing installation: tornado 6.1
Uninstalling tornado-6.1:
  Successfully uninstalled tornado-6.1
     |████████████████████████████████| 516 kB 4.2 MB/s 
  Created wheel for tornado: filename=tornado-5.1-cp37-cp37m-linux_x86_64.whl size=462844 sha256=46abe6733217c85e4ba05ea3d0f736a093353cae82a2eae4f102be6dd378f0fc
  Stored in directory: /root/.cache/pip/wheels/8e/53/de/6a6860ebf6d2958e20a9a8337f25b0369f1666002ade0ce284
Successfully built tornado
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-client 7.2.2 requires tornado>=6.0, but you have tornado 5.1 which is incompatible.
Mounted at /content/gdrive


In [6]:
from fastbook import *
from fastai.vision.widgets import *

In [35]:
key = os.environ.get('AZURE_SEARCH_KEY', 'BING_SEARCH_API_KEY')

In [8]:
cloud_types = 'cumulus cloud', 'stratus cloud', 'cirrus cloud'
path = Path('clouds')

In [9]:
if not path.exists():
    path.mkdir()
    for o in cloud_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o}')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [12]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

(#3) [None,None,None]

In [17]:
clouds = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [18]:
clouds = clouds.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)
dls = clouds.dataloaders(path)

In [19]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,1.541311,0.781851,0.302326,00:39


epoch,train_loss,valid_loss,error_rate,time
0,1.089366,0.715509,0.209302,00:39
1,1.046867,0.646763,0.174419,00:40
2,0.884108,0.630488,0.139535,00:40
3,0.841090,0.614772,0.151163,00:39


In [21]:
learn.export()

In [23]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [24]:
learn_inf = load_learner(path/'export.pkl')

In [20]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [26]:
img = PILImage.create(btn_upload.data[-1])

In [27]:
out_pl = widgets.Output()

In [28]:
pred, pred_idx, probs = learn_inf.predict(img)

In [29]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: cirrus cloud; Probability: 0.9921')